In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from env import api_key
import time

pd.set_option("display.max_rows", None, "display.max_columns", None)

pd.reset_option("display.max_rows", "display.max_columns")

import re

In [2]:
time.sleep(10)

In [6]:
for n,i in enumerate(range(10)):
    print(n)
    print(i+100)

0
100
1
101
2
102
3
103
4
104
5
105
6
106
7
107
8
108
9
109


In [8]:
match_list = pd.read_csv('chris_match.csv')

In [12]:
match = match_list['0'][0]

In [ ]:
from time import sleep
def get_json_list(match_list,api_key):
    url = 'https://americas.api.riotgames.com/'

    json_list = []

    for match,i in enumerate(match_list):
        if i%100 == 0:
            print('waiting')
            sleep(160)
            query = f'lol/match/v5/matches/{match}/timeline/?api_key={api_key}'
            query2 = f'lol/match/v5/matches/{match}/timeline/?api_key={api_key}'
            response = requests.get(url+query)
            json_list.append(response.json())
        else:
            query = f'lol/match/v5/matches/{match}/timeline/?api_key={api_key}'
            response = requests.get(url+query)
            json_list.append(response.json())
    return json_list

In [13]:
def prepare(jsonList, jsonList2,time):
    df = pd.DataFrame()
    
    for index, data in enumerate(jsonList):
        if ((data['info']['frames'][-1]['events'][-1]['timestamp']/60000) >= 20) & (str(jsonList2[index]['info']['gameMode'])=='CLASSIC'):
            final_d = {}
            kda = get_player_kda(data, time)
            final_d.update(kda)
            final_d.update(get_player_stats(data, time))
            final_d["gameMode"] = str(jsonList2[index]['info']['gameMode'])
            final_d["gameType"] = str(jsonList2[index]['info']['gameType'])
            final_d['gameVersion'] = str(jsonList2[index]['info']['gameVersion'])
            df = df.append(final_d, ignore_index=True)
            
            print(f"Finished with: {index} of {len(jsonList)}")
        else:
            print(f"Skipping: {index} due to <20 min or not classic")
    
    df = clean(df)
    return df

def clean(df):
    return df

def get_player_kda(data, time):
    df = pd.DataFrame()
    for index in range(len(data['info']['frames'])):
        for event in data['info']['frames'][index]['events']:
            if event['type'] == 'CHAMPION_KILL':
                df = df.append(event, ignore_index =True)
            

    df.timestamp = df.timestamp / 60_000

    kills_df = df[(df.type== 'CHAMPION_KILL') & (df.timestamp <= time)]

    d = {}
    mylist = []
    for value in kills_df.assistingParticipantIds:
        if type(value) == list:
            for assist in value:
                mylist.append(assist)
    temp = pd.DataFrame(mylist)

    for index, player in enumerate(kills_df.killerId.value_counts().sort_index()):

        # Grabbing kills and saving values to same dictionary
        d['killsplayer_'+str(int(kills_df.killerId.value_counts().sort_index().index[index]))] =\
        kills_df.killerId.value_counts().sort_index().iloc[index]

        # Grabbing deaths and saving values to same dictionary
    for index, player in enumerate(kills_df.victimId.value_counts().sort_index()):
        d['deathsplayer_'+str(int(kills_df.victimId.value_counts().sort_index().index[index]))] =\
        kills_df.victimId.value_counts().sort_index().iloc[index]

        # Grabbing assists and saving values to same dictionary
    for index, player in enumerate(temp[0].value_counts().sort_index()):
        d['assistsplayer_'+str(temp[0].value_counts().sort_index().index[index])] = temp.value_counts().sort_index().iloc[index]

    df = pd.DataFrame()
    for index in range(len(data['info']['frames'])):
        for event in data['info']['frames'][index]['events']:
            if event['type'] == 'ELITE_MONSTER_KILL':
                df = df.append(event, ignore_index =True)
            elif event['type']== 'GAME_END':
                df = df.append(event, ignore_index =True)

    kills_df = df[df.type == 'ELITE_MONSTER_KILL']
    for index, player in enumerate(kills_df[kills_df.monsterType=='DRAGON'].killerTeamId.value_counts().sort_index()):

        # Grabbing dragons and saving values to same dictionary
        d['dragon_team'+str(int(kills_df[kills_df.monsterType=='DRAGON'].killerTeamId.value_counts().sort_index().index[index]))] =\
        kills_df[kills_df.monsterType=='DRAGON'].killerTeamId.value_counts().sort_index().iloc[index]

    for index, player in enumerate(kills_df[kills_df.monsterType=='RIFTHERALD'].killerTeamId.value_counts().sort_index()):

        # Grabbing riftherald and saving values to same dictionary
        d['riftherald_team'+str(int(kills_df[kills_df.monsterType=='RIFTHERALD'].killerTeamId.value_counts().sort_index().index[index]))] =\
        kills_df[kills_df.monsterType=='RIFTHERALD'].killerTeamId.value_counts().sort_index().iloc[index]

    for index, player in enumerate(kills_df[kills_df.monsterType=='BARON_NASHOR'].killerTeamId.value_counts().sort_index()):

        # Grabbing baron and saving values to same dictionary
        d['baron_team'+str(int(kills_df[kills_df.monsterType=='BARON_NASHOR'].killerTeamId.value_counts().sort_index().index[index]))] =\
        kills_df[kills_df.monsterType=='BARON_NASHOR'].killerTeamId.value_counts().sort_index().iloc[index]

    # Grabbing target, winning team
    d['winningTeam'] = int(df[df.type == 'GAME_END'].winningTeam)

    df = pd.DataFrame()
    for index in range(len(data['info']['frames'])):
        for event in data['info']['frames'][index]['events']:
            if event['type'] == 'WARD_PLACED':
                df = df.append(event, ignore_index =True)

    kills_df = df[df.type == 'WARD_PLACED']
    for index, player in enumerate(kills_df[kills_df.type=='WARD_PLACED'].creatorId.value_counts().sort_index()):

        # Grabbing baron and saving values to same dictionary
        d['ward_player_'+str(int(kills_df[kills_df.type=='WARD_PLACED'].creatorId.value_counts().sort_index().index[index]))] =\
        kills_df[kills_df.type=='WARD_PLACED'].creatorId.value_counts().sort_index().iloc[index]
    df = pd.DataFrame()
    
    for index in range(len(data['info']['frames'])):
        for event in data['info']['frames'][index]['events']:
            if event['type'] == 'BUILDING_KILL':
                df = df.append(event, ignore_index =True)

    kills_df = df[df.type == 'BUILDING_KILL']

    for index, player in enumerate(kills_df[kills_df.buildingType =='TOWER_BUILDING'].teamId.value_counts().sort_index()):

        # Grabbing baron and saving values to same dictionary
        d['towers_lost_team'+str(int(kills_df[kills_df.buildingType =='TOWER_BUILDING'].teamId.value_counts().sort_index().index[index]))] =\
        kills_df[kills_df.buildingType =='TOWER_BUILDING'].teamId.value_counts().sort_index().iloc[index]
    
    for index, player in enumerate(kills_df[kills_df.buildingType =='INHIBITOR_BUILDING'].teamId.value_counts().sort_index()):

        # Grabbing baron and saving values to same dictionary
        d['inhibs_lost_team'+str(int(kills_df[kills_df.buildingType =='INHIBITOR_BUILDING'].teamId.value_counts().sort_index().index[index]))] =\
        kills_df[kills_df.buildingType =='INHIBITOR_BUILDING'].teamId.value_counts().sort_index().iloc[index]


    return d

def get_player_stats(data, time):
    player_stats = []

    #Here, each timeframe represents about one minute
    timeframe = data['info']['frames'][time]
    players = timeframe['participantFrames']
    #Now create a dicitonary to hold the players' stats from this timeframe
    players_dict = {}
    for i in range(1, len(players) + 1):
        #Now create a temp dict to store the current players stats
        #Use formatted strings to automatically increment the player label
        temp_dict = {
            f'currentGold_{i}': players[str(i)]['currentGold'],
            f'magicDamageDoneToChampions_{i}': players[str(i)]['damageStats']['magicDamageDoneToChampions'],
            f'physicalDamageDoneToChampions_{i}': players[str(i)]['damageStats']['physicalDamageDoneToChampions'],
            f'trueDamageDoneToChampions_{i}': players[str(i)]['damageStats']['trueDamageDoneToChampions'],
            f'totalDamageDoneToChampions_{i}': players[str(i)]['damageStats']['totalDamageDoneToChampions'],
            f'goldPerSecond_{i}': players[str(i)]['goldPerSecond'],
            f'jungleMinionsKilled_{i}': players[str(i)]['jungleMinionsKilled'],
            f'level_{i}': players[str(i)]['level'],
            f'minionsKilled_{i}': players[str(i)]['minionsKilled'],
            f'timeEnemySpentControlled_{i}': players[str(i)]['timeEnemySpentControlled'],
            f'totalGold_{i}': players[str(i)]['totalGold'],
            f'xp_{i}': players[str(i)]['xp']
            }
        #Now that I have the current players stats, extend it to the overall players_dict
        players_dict.update(temp_dict)
    #Update the players_dict one more time with the timestamp for the timeframe
    players_dict.update({'timestamp' : timeframe['timestamp']})
    #Append the players_dict to the overall player_stats list of dicts
    player_stats.append(players_dict)
    return player_stats[0]

In [19]:
df = pd.read_csv('chris_match.csv')
df = df.rename(columns={'0':"matchlist"})
match_ids = list(df.matchlist)
match_ids[10:15]

['NA1_4098742051',
 'NA1_4098623140',
 'NA1_4098608600',
 'NA1_4098575608',
 'NA1_4098098411']

In [20]:
jsonList=[]
jsonList2 = []

In [23]:
url = 'https://americas.api.riotgames.com/'

for x in range(90, 95):
    game = match_ids[x]
    
    query = f'lol/match/v5/matches/{game}/timeline/?api_key={api_key}'
    query2 = f'lol/match/v5/matches/{game}?api_key={api_key}'
    response = requests.get(url+query)
    response2 = requests.get(url+query2)
    jsonList.append(response.json())
    jsonList2.append(response2.json())

In [24]:
jsonList

[{'status': {'message': 'Forbidden', 'status_code': 403}},
 {'status': {'message': 'Forbidden', 'status_code': 403}},
 {'status': {'message': 'Forbidden', 'status_code': 403}},
 {'status': {'message': 'Forbidden', 'status_code': 403}},
 {'status': {'message': 'Forbidden', 'status_code': 403}},
 {'status': {'message': 'Forbidden', 'status_code': 403}},
 {'status': {'message': 'Forbidden', 'status_code': 403}},
 {'status': {'message': 'Forbidden', 'status_code': 403}},
 {'status': {'message': 'Forbidden', 'status_code': 403}},
 {'status': {'message': 'Forbidden', 'status_code': 403}}]

In [22]:
df = prepare(jsonList, jsonList2, 20)

KeyError: 'info'

In [ ]:
df = df.fillna(0)

In [ ]:
columns=['deathsplayer_',
'goldPerSecond_',
'jungleMinionsKilled_',
'killsplayer_',
'level_',
'magicDamageDoneToChampions_',
'minionsKilled_',
'physicalDamageDoneToChampions_',
'timeEnemySpentControlled_',
'totalDamageDoneToChampions_',
'totalGold_',
'trueDamageDoneToChampions_',
'ward_player_',
'assistsplayer_',
'xp_']

In [14]:
for col in columns:
    
    total = 0
    
    for i in range(1, 6):
        total += int(df.iloc[0][f'{col}{i}'])
    #df.iloc[0]
        df.iloc[0][f'team_{col}100'] = total

    total = 0
    
    for i in range(6, 11):
        total += int(df.iloc[0][f'{col}{i}'])
    #df.iloc[0]
        df.iloc[f'team_{col}200'] = total

NameError: name 'columns' is not defined